In [1]:
## Bibliotecas
import pandas as pd
import bokeh.plotting as bp
import sys
sys.path.append(r'C:\Users\giovanni.5683\GITHUB\controlling_postgreSQL')  # Adiciona o diretório onde está conn.py
from conn import get_connection
engine = get_connection()
cursor = engine.cursor()

In [2]:
## Conexão e leitura da view
Pendente = """SELECT * FROM byd.bonus_view"""
df_pendente = pd.read_sql_query(Pendente, engine)

C:\Users\giovanni.5683\AppData\Local\Temp\ipykernel_46712\3687575432.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pendente = pd.read_sql_query(Pendente, engine)


In [3]:
## Filtragem das colunas
df_pendente = df_pendente[['idnfsexterno','des_modelo', 'apontamento', 'competencia', 'bonus_utilizado', 'valor_bonus', 'trade', 'bonus_dpto', 'trade_mkt_dpto']]
df_pendente

,idnfsexterno,des_modelo,apontamento,competencia,bonus_utilizado,valor_bonus,trade,bonus_dpto,trade_mkt_dpto
0,A25519901V210,DOLPHIN 70KW Motor 405km - Afterglow Pink - An...,Certo!,Outubro,Equalização,14221.4,None,14221.4,None
1,A25252731V210,DOLPHIN 70KW Motor 405km - Afterglow Pink - An...,Certo!,Outubro,Equalização,14221.4,None,14221.4,None
2,A251250881V210,DOLPHIN 70KW Motor 405km - Afterglow Pink - An...,Certo!,Setembro,Equalização,9221.4,None,9221.4,None
3,A2583641V210,DOLPHIN 70KW Motor 405km - Afterglow Pink - An...,Certo!,Outubro,Equalização,14221.4,None,14221.4,None
4,A251214891V210,DOLPHIN 70KW Motor 405km - Afterglow Pink - An...,Revisar Divergência!,Julho,Varejo,17000.0,None,None,None
...,...,...,...,...,...,...,...,...,...
5227,A2517425516919VD,I/BYD DOLPHIN MINI,Certo!,Dezembro,Venda Direta,0.00,None,None,None
5228,A2517426517935VD,I/BYD DOLPHIN MINI,Certo!,Dezembro,Venda Direta,0.00,None,None,None
5229,A25111074517469VD,SONG PRO DMI GL,Certo!,Dezembro,Venda Direta,0.00,None,None,None
5230,A251208942516888VD,SONG PRO DMI GL,Certo!,Dezembro,Venda Direta,0.00,None,None,None


In [ ]:
## Conexão e leitura da view
Pendente = """SELECT * FROM byd.bonus_view"""
df_pendente = pd.read_sql_query(Pendente, engine)

## Filtragem dos dados 'Revisar Divergência!' para os meses de Agosto a Dezembro
df_pv = df_pendente[
    (df_pendente['apontamento'] == 'Revisar Divergência!') &
    (df_pendente['competencia'].isin(['Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']))
]
df_pv

C:\Users\giovanni.5683\AppData\Local\Temp\ipykernel_46712\3258002077.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pendente = pd.read_sql_query(Pendente, engine)


,idnfsexterno,revenda,empresa,numero_nota_fiscal,serie_nota_fiscal,dta_processamento,tipo_transacao,des_tipo_transacao,cod_cliente,nome_cliente,...,opt_comecial,campanha,detalhes,bonus_utilizado,bonus_dpto,rebate_dpto,ipva_dpto,trade_mkt_dpto,data_final_venda,competencia


In [5]:
## Atualização dos dados na tabela byd.controladoria
for index, row in df_pv.iterrows():
    idnfsexterno = row['idnfsexterno']
    valor_bonus = row['valor_bonus']
    trade = row['trade']
    
    update_query = """
    UPDATE byd.controladoria 
    SET bonus_dpto = %s, trade_mkt_dpto = %s 
    WHERE idnfsexterno = %s
    """
    cursor.execute(update_query, (valor_bonus, trade, idnfsexterno))

engine.commit()